이미지데이터와 라벨을 분류하기 위한 CustomDataset 클래스

In [11]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset
from torchvision import transforms

class CustomDataset(Dataset):
    def __init__(self, root_dir, mode='Training', transform=None):
        self.root_dir = root_dir
        self.mode = mode
        self.transform = transform

        # 이미지 및 라벨 리스트 초기화
        self.images = []
        self.labels = []

        # 데이터 폴더 경로 설정
        data_folder = 'Training' if mode == 'Training' else 'Validation'
        data_path = os.path.join(root_dir, data_folder)

        # 이미지와 라벨 로드
        self._load_data(data_path)

    #주어진 데이터 폴더에서 이미지와 라벨 로드
    def _load_data(self, data_path): 
        label_folder_path = os.path.join(data_path, "라벨링데이터", "05.상추")
        image_folder_path = os.path.join(data_path, "원천데이터", "05.상추")

        label_folders = os.listdir(label_folder_path)

        for label_folder in label_folders:
            label = int(label_folder.split(".")[0].split()[-1])  # 라벨 추출
            label_path = os.path.join(label_folder_path, label_folder)
            image_path = os.path.join(image_folder_path, label_folder)

            # 이미지 파일과 라벨 파일 매칭
            for image_filename in os.listdir(image_path):
                image_file_path = os.path.join(image_path, image_filename)
                self.images.append(image_file_path)
                self.labels.append(label)

    #데이터 셋의 총 데이터수
    def __len__(self):
        return len(self.images)
    
    #주어진 인덱스에 해당하는 라벨과 이미지 출력 
    def __getitem__(self, idx):
        image_path = self.images[idx]
        label = self.labels[idx]
        
        # 이미지 불러오기
        image = Image.open(image_path).convert('RGB')
        
        # 이미지 전처리 적용
        if self.transform:
            image = self.transform(image)
        
        return image, label

# 데이터셋의 폴더 경로 설정
root_dir = "/Volumes/T7/상추 질병 진단/data"

# 데이터 전처리를 위한 변환 정의
transform = transforms.Compose([
    transforms.Resize((224, 224)), #resnet이 훈련될때 224 * 224를 사용하기 때문에 조정
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 데이터셋 인스턴스 생성
train_dataset = CustomDataset(root_dir, mode='Training', transform=transform)
val_dataset = CustomDataset(root_dir, mode='Validation', transform=transform)

# 데이터로더 생성
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)

In [48]:
import os
import json
from torch.utils.data import Dataset
from PIL import Image
from torchvision import transforms
import torch

class CustomDataset(Dataset):
    def __init__(self, root_dir, mode='Training', transform=None):
        self.root_dir = root_dir
        self.mode = mode
        self.transform = transform

        self.images = []
        self.labels = []

        data_folder = 'Training' if mode == 'Training' else 'Validation'
        data_path = os.path.join(root_dir, data_folder)

        self._load_data(data_path)

    def _load_data(self, data_path):
        image_folder_path = os.path.join(data_path, "원천데이터", "05.상추")
        label_folder_path = os.path.join(data_path, "라벨링데이터", "05.상추")

        image_folders = os.listdir(image_folder_path)

        for image_folder in image_folders:
            image_path = os.path.join(image_folder_path, image_folder)
            label_path = os.path.join(label_folder_path, image_folder)

            for image_filename in os.listdir(image_path):
                image_file_path = os.path.join(image_path, image_filename)
                label_file_path = os.path.join(label_path, image_filename.replace('.jpeg', '.json'))

                # JSON 파일에서 disease 값을 읽어서 라벨 결정
                if os.path.exists(label_file_path):
                    with open(label_file_path, 'r') as f:
                        label_info = json.load(f)
                        disease = label_info['annotations']['disease']

                        if disease == 9:  # 균핵병
                            label = 1
                        elif disease == 10:  # 노균병
                            label = 2
                        else:  # 정상
                            label = 0
                else:
                    label = 0  # 라벨 정보가 없는 경우 정상으로 처리

                self.images.append(image_file_path)
                self.labels.append(label)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = self.images[idx]
        label = self.labels[idx]

        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)

        return image, label


In [50]:
import torch
import torch.nn as nn #신경망

class BasicBlock(nn.Module): #모듈정의
  def __init__(self, in_channels, out_channels, kernel_size=3): #입력채널수 , 출력채널수 , 합성곱이 갖는 커널의 크기
    super(BasicBlock, self).__init__() #nn.Module의 요소를 가져와라

    self.in_channels = in_channels
    self.out_channels = out_channels

    # 합성곱층 정의
    self.c1 = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=1)
    self.c2 = nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size, padding=1) # c1의 출력을 입력으로 받는다.

    # 다운샘플링이 필요한 경우에만 다운샘플 레이어 정의
    self.downsample = nn.Conv2d(in_channels, out_channels, kernel_size=1) if in_channels != out_channels else None

    # 합성곱 하나 거칠때마다 배치정규화도 필요
    self.bn1 = nn.BatchNorm2d(num_features=out_channels)
    self.bn2 = nn.BatchNorm2d(num_features=out_channels)

    # 활성화 함수 정의
    self.relu = nn.ReLU()

  # 순전파 정의
  def forward(self, x):
    identity = x

    # Resnet의 기본 블록에서 F(x) 부분
    x = self.c1(x) # 합성곱
    x = self.bn1(x) # 배치 정규화
    x = self.relu(x) # 활성화
    x = self.c2(x)
    x = self.bn2(x)

    # 입력과 출력 채널이 다를 경우에만 다운샘플 적용
    if self.downsample is not None:
      identity = self.downsample(identity)

    x += identity # 은닉층을 거친 x + 다운샘플을 거친 identity
    x = self.relu(x) 

    return x


In [54]:
class ResNet(nn.Module):
    def __init__(self, num_classes=3):  # 3개의 클래스로 수정
        super(ResNet, self).__init__()

        # 기본 블록 3
        self.b1 = BasicBlock(in_channels=3, out_channels=64)
        self.b2 = BasicBlock(in_channels=64, out_channels=128)
        self.b3 = BasicBlock(in_channels=128, out_channels=256)
       

        # 평균값 풀링
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)

        # 분류기 층
        self.fc1 = nn.Linear(in_features=4096, out_features=2048) #4*4*256 -> 원래 32*32크기 이미지가 4*4까지 떨어지게 됐기 때문
        self.fc2 = nn.Linear(in_features=2048, out_features=512)
        self.fc3 = nn.Linear(in_features=512, out_features=num_classes)


        self.relu = nn.ReLU() #활성화 함수

      #순전파
    def forward(self, x):
    #기본블록과 풀링층 통과
       x = self.b1(x) #첫번째 블록
       x = self.pool(x) #풀링
       x = self.b2(x)
       x = self.pool(x)
       x = self.b3(x)
       x = self.pool(x) #여기까지는 2차원 이미지

       print("크기 출력" + x.shape) 
    #분류기의 입력으로 사용하기 위한 평탄화
       x = torch.flatten(x, start_dim=1)
 
    #분류기로 예측값 출력
       x = self.fc1(x) #분류기
       x = self.relu(x) #활성화
       x = self.fc2(x)
       x = self.relu(x)
       x = self.fc3(x) #결국 10개를 갖게 된다.

       return x

In [52]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# 데이터셋과 데이터로더
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# 모델 인스턴스 생성
model = ResNet(num_classes=3)

# 손실 함수 및 최적화기
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


# 장치 설정 (GPU 사용 가능 시)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)



ResNet(
  (b1): BasicBlock(
    (c1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (c2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (downsample): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1))
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (b2): BasicBlock(
    (c1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (c2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (downsample): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (b3): BasicBlock(
    (c1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [53]:
# 학습 루프
num_epochs = 10
for epoch in range(num_epochs):
    model.train()  # 모델을 훈련 모드로 설정
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)  # 데이터를 GPU로 이동
        optimizer.zero_grad()  # 그래디언트 초기화
        outputs = model(images)  # 모델 순전파
        loss = criterion(outputs, labels)  # 손실 계산 - 크로스 엔트로피 
        loss.backward()  # 역전파
        optimizer.step()  # 매개변수 업데이트
        running_loss += loss.item() * images.size(0)
    epoch_loss = running_loss / len(train_dataset)
    
    # 검증 데이터셋을 사용하여 모델 평가
    model.eval()  # 모델을 평가 모드로 설정
    val_loss = 0.0
    correct_preds = 0
    total_preds = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)  # 데이터를 GPU로 이동
            outputs = model(images)  # 모델 순전파
            loss = criterion(outputs, labels)  # 손실 계산
            val_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            correct_preds += (predicted == labels).sum().item()
            total_preds += labels.size(0)
    val_loss /= len(val_dataset)
    val_acc = correct_preds / total_preds
    
    # 에포크마다 결과 출력
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")


torch.Size([32, 256, 28, 28])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x200704 and 4096x2048)

In [9]:
import os

def get_class_labels(dataset_root):
    # 클래스 레이블 딕셔너리 초기화
    class_labels = {}

    # 데이터셋 루트 디렉토리 내의 하위 디렉토리(클래스별로 구분된 폴더) 탐색
    for class_name in os.listdir(dataset_root):
        class_path = os.path.join(dataset_root, class_name)
        if os.path.isdir(class_path):
            # 클래스 레이블 딕셔너리에 클래스명과 해당 클래스의 인덱스 추가
            class_labels[class_name] = len(class_labels)

    return class_labels

# 데이터셋 루트 디렉토리 경로 설정
dataset_root = '/Volumes/T7/상추 질병 진단/data'

# 클래스 레이블 확인
class_labels = get_class_labels(dataset_root)
print("클래스 레이블:", class_labels)


클래스 레이블: {'Validation': 0, 'Training': 1}


In [17]:
import os

def get_class_labels(dataset_root):
    # 클래스 레이블 딕셔너리 초기화
    class_labels = {}

    # 데이터셋 루트 디렉토리 내의 하위 디렉토리(클래스별로 구분된 폴더) 탐색
    for class_name in os.listdir(dataset_root):
        class_path = os.path.join(dataset_root, class_name)
        if os.path.isdir(class_path):
            # 클래스 레이블 딕셔너리에 클래스명과 해당 클래스의 인덱스 추가
            class_labels[class_name] = len(class_labels)

    return class_labels

# 데이터셋 루트 디렉토리 경로 설정
dataset_root = '/Volumes/T7/상추 질병 진단/data/Training/라벨링데이터/05.상추'

# 클래스 레이블 확인
class_labels = get_class_labels(dataset_root)
print("클래스 레이블:", class_labels)

클래스 레이블: {'05.상추_0.정상': 0, '05.상추_1.질병': 1, '05.상추_9.증강': 2}


In [1]:
#데이터 로더중 none 값을 걸래는 dataset
import os
import json
from torch.utils.data import Dataset, DataLoader
from PIL import Image, UnidentifiedImageError
from torchvision import transforms
import torch

class CustomDataset(Dataset):
    def __init__(self, root_dir, mode='Training', transform=None):
        self.root_dir = root_dir
        self.mode = mode
        self.transform = transform

        self.images = []
        self.labels = []

        data_folder = 'Training' if mode == 'Training' else 'Validation'
        data_path = os.path.join(root_dir, data_folder)

        self._load_data(data_path)

    def _load_data(self, data_path):
        image_folder_path = os.path.join(data_path, "원천데이터", "05.상추")
        label_folder_path = os.path.join(data_path, "라벨링데이터", "05.상추")

        image_folders = os.listdir(image_folder_path)

        for image_folder in image_folders:
            image_path = os.path.join(image_folder_path, image_folder)
            label_path = os.path.join(label_folder_path, image_folder)

            for image_filename in os.listdir(image_path):
                image_file_path = os.path.join(image_path, image_filename)
                label_file_path = os.path.join(label_path, image_filename.replace('.jpeg', '.json'))

                # JSON 파일에서 disease 값을 읽어서 라벨 결정
                if os.path.exists(label_file_path):
                    with open(label_file_path, 'r') as f:
                        label_info = json.load(f)
                        disease = label_info['annotations']['disease']

                        if disease == 9:  # 균핵병
                            label = 1
                        elif disease == 10:  # 노균병
                            label = 2
                        else:  # 정상
                            label = 0
                else:
                    label = 0  # 라벨 정보가 없는 경우 정상으로 처리

                self.images.append(image_file_path)
                self.labels.append(label)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = self.images[idx]
        label = self.labels[idx]

        try:
            image = Image.open(image_path).convert('RGB')
            if self.transform:
                image = self.transform(image)
        except (IOError, UnidentifiedImageError) as e:
            print(f"Warning: Could not read image {image_path}. Skipping. Error: {e}")
            return None, None

        return image, label

# DataLoader에서 None 타입 데이터를 걸러내기 위한 collate_fn
def my_collate_fn(batch):
    batch = list(filter(lambda x: x[0] is not None, batch))
    return torch.utils.data.DataLoader.default_collate(batch)

# CustomDataset과 DataLoader 인스턴스 생성 예시
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

dataset = CustomDataset(root_dir="/Volumes/T7/상추 질병 진단/data", mode='Training', transform=transform)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=my_collate_fn)


In [5]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models, transforms
from torchvision.models.resnet import ResNet50_Weights  # 추가된 부분

# 데이터 변환 정의
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet50에 맞는 이미지 크기로 조정
    transforms.ToTensor(),          # 이미지를 Tensor로 변환
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet 평균 및 표준편차로 정규화
                         std=[0.229, 0.224, 0.225])
])

# CustomDataset 인스턴스 생성
root_dir = '/Volumes/T7/상추 질병 진단/data'  # 데이터셋 경로 지정
train_dataset = CustomDataset(root_dir=root_dir, mode='Training', transform=transform)
val_dataset = CustomDataset(root_dir=root_dir, mode='Validation', transform=transform)

# DataLoader 생성
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# ResNet50 모델 로드 및 수정
weights = ResNet50_Weights.DEFAULT  # 사전 훈련된 가중치를 지정하는 부분
model = models.resnet50(weights=weights)  # 수정된 부분
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 3)  # 마지막 레이어를 3개 클래스 출력으로 교체

# 손실 함수 및 최적화 알고리즘 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# GPU 사용 설정
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

# 훈련 루프
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

    # 검증 루프
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    print(f'Accuracy on validation set: {100 * correct / total}%')


KeyboardInterrupt: 

In [6]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models, transforms

# 데이터 변환 정의
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet50에 맞는 이미지 크기로 조정
    transforms.ToTensor(),          # 이미지를 Tensor로 변환
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet 평균 및 표준편차로 정규화
                         std=[0.229, 0.224, 0.225])
])

# CustomDataset 인스턴스 생성
root_dir = '/Volumes/T7/상추 질병 진단/data'  # 데이터셋 경로 지정
train_dataset = CustomDataset(root_dir=root_dir, mode='Training', transform=transform)
val_dataset = CustomDataset(root_dir=root_dir, mode='Validation', transform=transform)

# DataLoader 생성
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# ResNet50 모델 로드 및 수정
model = models.resnet50(pretrained=True)  # 이미지넷 데이터셋에서 사전 훈련된 가중치를 사용하여 초기화
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 3)  # 마지막 레이어를 3개 클래스 출력으로 교체

# 손실 함수 및 최적화 알고리즘 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# GPU 사용 설정
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

# 훈련 루프
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

    # 검증 루프
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    print(f'Accuracy on validation set: {accuracy}%')


/Users/leeyeonju/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/leeyeonju/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


KeyboardInterrupt: 

In [ ]:
#데이터 로더중 none 값을 걸래는 dataset
import os
import json
from torch.utils.data import Dataset, DataLoader
from PIL import Image, UnidentifiedImageError
from torchvision import transforms
import torch

class CustomDataset(Dataset):
    def __init__(self, root_dir, mode='Training', transform=None):
        self.root_dir = root_dir
        self.mode = mode
        self.transform = transform

        self.images = []
        self.labels = []

        data_folder = 'Training' if mode == 'Training' else 'Validation'
        data_path = os.path.join(root_dir, data_folder)

        self._load_data(data_path)

    def _load_data(self, data_path):
        image_folder_path = os.path.join(data_path, "원천데이터", "05.상추")
        label_folder_path = os.path.join(data_path, "라벨링데이터", "05.상추")

        image_folders = os.listdir(image_folder_path)

        for image_folder in image_folders:
            image_path = os.path.join(image_folder_path, image_folder)
            label_path = os.path.join(label_folder_path, image_folder)

            for image_filename in os.listdir(image_path):
                image_file_path = os.path.join(image_path, image_filename)
                label_file_path = os.path.join(label_path, image_filename.replace('.jpeg', '.json'))

                # JSON 파일에서 disease 값을 읽어서 라벨 결정
                if os.path.exists(label_file_path):
                    with open(label_file_path, 'r') as f:
                        label_info = json.load(f)
                        disease = label_info['annotations']['disease']

                        if disease == 9:  # 균핵병
                            label = 1
                        elif disease == 10:  # 노균병
                            label = 2
                        else:  # 정상
                            label = 0
                else:
                    label = 0  # 라벨 정보가 없는 경우 정상으로 처리

                self.images.append(image_file_path)
                self.labels.append(label)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = self.images[idx]
        label = self.labels[idx]

        try:
            image = Image.open(image_path).convert('RGB')
            if self.transform:
                image = self.transform(image)
        except (IOError, UnidentifiedImageError) as e:
            print(f"Warning: Could not read image {image_path}. Skipping. Error: {e}")
            return None, None

        return image, label

# DataLoader에서 None 타입 데이터를 걸러내기 위한 collate_fn
def my_collate_fn(batch):
    batch = list(filter(lambda x: x[0] is not None, batch))
    return torch.utils.data.DataLoader.default_collate(batch)



In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models, transforms

# 데이터 변환 정의
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet50에 맞는 이미지 크기로 조정
    transforms.ToTensor(),          # 이미지를 Tensor로 변환
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet 평균 및 표준편차로 정규화
                         std=[0.229, 0.224, 0.225])
])

# CustomDataset 인스턴스 생성
root_dir = '/Volumes/T7/상추 질병 진단/smalldata'  # 데이터셋 경로 지정
train_dataset = CustomDataset(root_dir=root_dir, mode='Training', transform=transform)
val_dataset = CustomDataset(root_dir=root_dir, mode='Validation', transform=transform)

# DataLoader 생성
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# ResNet50 모델 로드 및 수정
model = models.resnet50(pretrained=True)  # 이미지넷 데이터셋에서 사전 훈련된 가중치를 사용하여 초기화
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 3)  # 마지막 레이어를 3개 클래스 출력으로 교체

# 손실 함수 및 최적화 알고리즘 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# GPU 사용 설정
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

# 훈련 루프
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

    # 검증 루프
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    print(f'Accuracy on validation set: {accuracy}%')